In [155]:
# https://www.bitaddress.org/
# https://en.wikipedia.org/wiki/Base58

BASE58_CHARS = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'


def map_base58():
    '''Create a map of chars for Base58'''
    tmp_dir = {}
    i = 0
    for c in BASE58_CHARS:
        tmp_dir[c] = i
        i = i + 1
    return tmp_dir


BASE58_MAP = map_base58()


def base58_enc(in_num):
    '''Encode a number into Bitcoin Base58 format'''
    if in_num < 0:
        raise Exception("Positive integers only")
    tmp_out = []
    while in_num > 0:
        tmp_out.append(BASE58_CHARS[in_num % 58])
        in_num = in_num // 58
    tmp_out.reverse()
    return ''.join(tmp_out)


def base58_dec(in_str):
    '''Decode a Bitcoin Base58 string to a number'''
    out_num = 0
    for c in in_str:
        if c not in BASE58_MAP:
            raise Exception("Invalid char, not in Base58: %c" % c)
        out_num = out_num*58 + BASE58_MAP[c]
    return out_num


def base256_to_base58(raw_str):
    '''Convert a raw string to Base58 string'''
    leading_zeros = 0
    num = 0
    for c in raw_str:
        if c != 0:
            break
        leading_zeros = leading_zeros + 1
    for c in raw_str:
        num = num*256 + c
    base58_str = ''.join(leading_zeros*['1']) + base58_enc(num)
    return base58_str


def base58_to_base256(base58_str):
    '''Convert a Base58 string to raw string'''
    leading_ones = 0
    for c in base58_str:
        if c != '1':
            break
        leading_ones = leading_ones + 1
    num = base58_dec(base58_str)
    raw_str_suffix_hex = '%02x' % num
    if len(raw_str_suffix_hex) % 2:
        raw_str_suffix_hex = '0%s' % raw_str_suffix_hex
    raw_str_hex = ''.join(leading_ones*['00']) + raw_str_suffix_hex
    raw_str = unhexlify(raw_str_hex)
    return raw_str

In [164]:
# https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses
# https://github.com/warner/python-ecdsa
# https://docs.python.org/3/library/hashlib.html
# Terminal: openssl ecparam -list_curves | grep -i secp256k1

import hashlib
from ecdsa import SigningKey, SECP256k1
from binascii import hexlify, unhexlify


def pub_key_from_priv_key(priv_key_hex):
    '''Obtain public key from the private key'''
    secexp = int(priv_key_hex, 16)
    signing_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
    sk_hex = hexlify(signing_key.to_string())
    vk = signing_key.get_verifying_key()
    vk_hex = hexlify(vk.to_string())
    pub_key_hex = vk_hex.decode('utf-8')
    return pub_key_hex


def bitcoin_addr_from_priv_key(priv_key_hex):
    '''Create Bitcoin address from the private key'''
    # Step 0: Having a private ECDSA key
    pub_key_hex = pub_key_from_priv_key(priv_key_hex)
    return bitcoin_addr_from_pub_key(pub_key_hex)


def bitcoin_addr_from_pub_key(pub_key_hex):
    '''Create Bitcoin address from the public key'''
    # Step 1: Take the corresponding public key generated with it
    # (65 bytes, 1 byte 0x04, 32 bytes corresponding to X coordinate,
    # 32 bytes corresponding to Y coordinate)
    full_pub_key_hex = '04%s' % pub_key_hex
    full_pub_key = unhexlify(full_pub_key_hex)
    assert len(full_pub_key) == 65

    # Step 2: Perform SHA-256 hashing on the public key
    sha256_hasher = hashlib.sha256(full_pub_key)

    # Step 3: Perform RIPEMD-160 hashing on the result of SHA-256
    ripemd160_hasher = hashlib.new('ripemd160')
    ripemd160_hasher.update(sha256_hasher.digest())

    # Step 4: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
    mainnet_format_hex = '00%s' % ripemd160_hasher.hexdigest()
    mainnet_format = unhexlify(mainnet_format_hex)

    # Step 5: Perform SHA-256 hash on the extended RIPEMD-160 result
    sha256_hasher = hashlib.sha256(mainnet_format)

    # Step 6: Perform SHA-256 hash on the result of the previous SHA-256 hash
    sha256_hasher2 = hashlib.sha256(sha256_hasher.digest())

    # Step 7: Take the first 4 bytes of the second SHA-256 hash. This is the address checksum
    checksum_4bytes = sha256_hasher2.digest()[:4]

    # Step 8: Add the 4 checksum bytes from stage 7 at the end of
    # extended RIPEMD-160 hash from stage 4.
    # This is the 25-byte binary Bitcoin Address.
    bitcoin_addr_raw = mainnet_format + checksum_4bytes
    assert len(bitcoin_addr_raw) == 25
    bitcoin_addr_hex = hexlify(bitcoin_addr_raw)

    # Step 9: Convert the result from a byte string into a base58 string using Base58Check encoding.
    # This is the most commonly used Bitcoin Address format
    bitcoin_addr = base256_to_base58(bitcoin_addr_raw)
    bitcoin_addr_raw_again = base58_to_base256(bitcoin_addr)
    assert bitcoin_addr_raw_again == bitcoin_addr_raw
    return bitcoin_addr


def verify_bitcoin_addr(bitcoin_addr):
    '''Verify Bitcoin address'''
    bitcoin_addr_raw = base58_to_base256(bitcoin_addr)
    assert len(bitcoin_addr_raw) == 25
    mainnet_format, checksum_4bytes = bitcoin_addr_raw[:21], bitcoin_addr_raw[21:]

    # Verify that the double SHA-256 has the same prefix as checksum_4bytes
    sha256_hasher = hashlib.sha256(mainnet_format)
    sha256_hasher2 = hashlib.sha256(sha256_hasher.digest())
    assert checksum_4bytes == sha256_hasher2.digest()[:4]
    
    return True

In [165]:
TEST_CASES = [
    {
        'priv': '18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725',
        'addr': '16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM',
    },
]

'''
{
    'priv': '80B1D685BAE45AF1D33231F47A8BC6F54CECC7DE76C15D60D658B38603346BD6910193D04B08',
    'addr': '1D2Gme2513ncWsxB4DchzT3ukeNUXYVv3c',
},
'''

for test_case in TEST_CASES:
    priv_key_hex = test_case['priv']
    bitcoin_addr = bitcoin_addr_from_priv_key(priv_key_hex)
    assert bitcoin_addr == test_case['addr']
    assert verify_bitcoin_addr(bitcoin_addr)